In [ ]:
# standard python utilities
import os
from os.path import join, basename,dirname
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# import seaborn as sns

# standard geospatial python utilities
# import pyproj # for converting proj4string
import shapely
import geopandas as gpd

from osgeo import gdal # need to load before rasterio because of issue

import rasterio

# mapping utilities
# import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.ticker import MaxNLocator

import flopy

In [ ]:
git_dir = os.getcwd()
while basename(git_dir) != 'GitHub':
    git_dir = dirname(git_dir)
usr_dir = os.getcwd()
while basename(usr_dir) != 'Users':
    temp = basename(usr_dir)
    usr_dir = dirname(usr_dir)
usr_dir += '/'+temp

py_dir = git_dir +'/CosumnesRiverRecharge/python_utilities/'
## Set up directory referencing
# Package data
gwfm_dir = usr_dir+'/Box/research_cosumnes/GWFlowModel/'
proj_dir = gwfm_dir+'Oneto_Denier/'
dat_dir = proj_dir+'Stream_level_data/'
fig_dir = proj_dir+'/Streambed_seepage/figures/'



In [ ]:
hob_dir = join(gwfm_dir, 'HOB_data')
dem_dir = gwfm_dir+ '/DEM_data'

In [ ]:
loadpth =  'C:/WRDAPP/GWFlowModel/Cosumnes/Stream_seepage'

model_nam = 'inset_oneto_denier'
model_ws = join(loadpth,model_nam)

m = flopy.modflow.Modflow.load('MF.nam', model_ws= model_ws, 
                                exe_name='mf-owhm.exe', version='mfnwt')


In [ ]:
grid_dir = join(gwfm_dir, 'DIS_data/streambed_seepage/grid')
grid_fn = join(grid_dir, model_nam,'rm_only_grid.shp')
grid_p = gpd.read_file(grid_fn)
grid_p.crs='epsg:32610'
m_domain = gpd.GeoDataFrame(pd.DataFrame([0]), geometry = [grid_p.unary_union], crs='epsg:32610')

In [ ]:
sfrdf = pd.DataFrame(m.sfr.reach_data)
grid_sfr = grid_p.set_index(['row','column']).loc[list(zip(sfrdf.i+1,sfrdf.j+1))].reset_index(drop=True)
grid_sfr = pd.concat((grid_sfr,sfrdf),axis=1)

In [ ]:
map_dir = gwfm_dir+'/Mapping/'
df = pd.read_csv(join(hob_dir,'CosumnesRiverPreserve_MW_screened_interval.csv'))
rm_sp = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Longitude,df.Latitude), crs='epsg:4326')
rm_sp = rm_sp.to_crs('epsg:32610')
rm_sp = rm_sp.rename(columns={'Well ID':'Sensor'})

# prepare output for modelgrid join
rm_t = rm_sp[rm_sp['At Oneto-Denier']=='Yes']


# find HOB well grid cell
rm_grid = gpd.sjoin(rm_t, grid_p)

# get model layer for heads
hob_row = rm_grid.row.values-1
hob_col = rm_grid.column.values-1

In [ ]:
point = list(zip(rm_t.geometry.x.values,rm_t.geometry.y.values))

raster_name = dem_dir+'/mwt_peri_2_3.tif/mwt_peri_2_3_clipped.tif'
with rasterio.open(raster_name) as src:
    rm_t['z_OD_m'] = [sample[0] for sample in src.sample(point)]

raster_name = gwfm_dir+"/DEM_data/USGS_ten_meter_dem/modeldomain_10m_transformed.tif"
with rasterio.open(raster_name) as src:
    rm_t['z_USGS_m'] = [sample[0] for sample in src.sample(point)]


In [ ]:
pd.DataFrame(rm_t).plot(x='Sensor', y=['MPE (meters)', 'z_OD_m','z_USGS_m'])
# rm_t

## Zonal stats on 2m DEM

In [ ]:
from rasterstats import zonal_stats

In [ ]:
fn = proj_dir+'/local_grid_elevation_m_statistics.shp'
if exists(fn):
    zs_df = gpd.read_file(fn)
    zs_df[['row','column']]-=1
else:
    raster_name = dem_dir+'/mwt_peri_2_3.tif/mwt_peri_2_3_clipped.tif'
    dem = rasterio.open(raster_name)
    # affine = dem.affine # didn't work
    affine = dem.meta['transform']

    stats = ['mean', 'median', 'majority','std','min','max']

    # takes several minutes
    zs_grid = zonal_stats(grid_p, raster=raster_name, stats=stats)
    # convert to dataframe
    zs_df = pd.DataFrame(zs_grid)
    # join zone stats of DEM to parcel data
    zs_df = grid_p.join(zs_df)
    # save to shapefile
    zs_df.to_file(proj_dir+'/local_grid_elevation_m_statistics.shp')
    zs_df[['row','column']]-=1

In [ ]:
# zs_df.plot('mean',legend=True)


In [ ]:
dem_local = np.zeros((m.dis.nrow,m.dis.ncol))
dem_local[zs_df.row-1, zs_df.column-1] = zs_df['mean']

In [ ]:
vmin = np.nanmin((dem_local, m.dis.top.array))
vmax = np.nanmax((dem_local, m.dis.top.array))

In [ ]:

fig,ax=plt.subplots(1,2)
ax[0].imshow(dem_local, vmin=vmin,vmax=vmax)
ax[0].set_title('Local DEM')
ax[1].set_title('Regional DEM')
im = ax[1].imshow(m.dis.top.array, vmin=vmin,vmax=vmax)
cbar_ax = fig.add_axes([0.95, 0.25, 0.04, 0.4])
fig.colorbar(im, cax=cbar_ax)

In [ ]:
plt.imshow(m.dis.top.array - dem_local, vmax=2)
plt.colorbar()

The local DEM has very similar trends and magnitude to the regional DEM yet with better channel and drainage definition which explains why the channel appears so much lower for the local DEM.

In [ ]:
grid_sfr

In [ ]:
dem_stat = gpd.read_file(join(gwfm_dir,'DIS_data/grid_elevation_m_statistics.shp'))
dem_stat[['row','column']]-=1

grid_reg = grid_sfr.join(dem_stat.set_index(['row','column'])[['mean','percentile','percenti10']], on=['i','j'])
grid_loc = grid_sfr.join(zs_df.set_index(['row','column'])[['mean','min','max']], on=['i','j'])


In [ ]:
fig,ax=plt.subplots()
grid_reg.plot(y=['strtop','mean','percentile','percenti10'],kind='line', ax=ax)
grid_loc.plot(y=['strtop','mean','min','max'],kind='line',style='--',ax=ax)

(grid_sfr.strtop/0.3048).plot(ax=ax)

Plotting the zonal stats on the local scale shows that the elevation minimums for all sfr grid cells are much higher than what was predicted by the cross-section sampling which means that the cross-section might be accidentally pulling lower points at further distances. I multiplied the minimums sampled by 3.28084 and found that I hadn't removed the conversion from ft to meters in the XS sampling code which is what is causing the error.

In [ ]:
fp_logger = pd.read_csv(join(gwfm_dir,'LAK_data','floodplain_logger_metadata.csv'))
fp_logger = gpd.GeoDataFrame(fp_logger, geometry = gpd.points_from_xy(fp_logger.Easting, fp_logger.Northing), crs='epsg:32610')
# find grid cell it is within
fp_grid = gpd.sjoin(fp_logger, grid_p, how='left',predicate='within')

In [ ]:
fp_grid['local_dem'] = dem_local[fp_grid.row-1,fp_grid.column-1]
fp_grid['regional_dem'] = m.dis.top.array[fp_grid.row-1,fp_grid.column-1]

Plotting the mean elevation from the local versus the preset elevations of the floodplain loggers shows that it is actually higher than expected. So the 2m DEM isn't actually too low in terms of height. The issue for the strm top may lie in how the raster is sampled and the minimum is chosen.

In [ ]:
fp_grid[fp_grid['Logger Type']=='River'].plot(x = 'Logger Location', y=['Elevation','local_dem','regional_dem'],kind='line')
plt.xticks(rotation=90)